In [33]:
-- Explore the data in the table
SELECT * 
FROM online_retail.csv
LIMIT (5);



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850,United Kingdom


Having run some code above to explore the data, will decide on a few questions to answer as if this data was from my business: 
1. How much money did the top ten customers spend & which countries did they come from? 
2. Did I spend more than 1% of my revenue on postage? (Assumes that unit price is what I made and there were no associated costs)
3. How many different products were included in the average order? 

The next code section will include the code to answer this question: 
1. How much money did the top ten customers spend & which countries did they come from? 

In [34]:
SELECT SUM(unitprice) AS total_profit, customerID, country
FROM online_retail.csv
GROUP BY customerID, country 
HAVING customerID IS NOT NULL
ORDER BY total_profit DESC
LIMIT (10);

,total_profit,CustomerID,Country
0,41376.33,14096,United Kingdom
1,40278.90,15098,United Kingdom
2,31060.66,14911,EIRE
3,25108.89,12744,Singapore
4,24111.14,16029,United Kingdom
5,20333.18,17841,United Kingdom
6,15115.60,12748,United Kingdom
7,13255.22,12536,France
8,8035.24,12757,Portugal
9,7839.02,14606,United Kingdom


The above table shows that the top customer spent £41,376.33, and was from the UK. The top two customers both spent > £40k.

6 of the top 10 spending customers were from the UK, with one customer from each of Ireland, Singapore, France and Portugal. (Since this isn't really my dataset, it would be reasonable to assume that this business is UK-based, thus why I have used GBP as the currency). 


The next section of code works on the question below
2. Did I spend more than 1% of my revenue on postage? (This assumes that unit price is what I made and there were no associated costs besides 'post', which is excluded from the profits.)

In [35]:
SELECT SUM(UnitPrice)
FROM online_retail.csv
-- Note that stock code = 'POST' indicates the value spent on postage 
WHERE StockCode = 'POST';

,sum(UnitPrice)
0,46388.47


The above code and query result showed that the total 'unit price' for all the POST costs was £46,388.47 (i.e. this is what was spent on postage). 

Next, I identified the total money made given that the stock code is not 'POST'. Note that this assumes any other stock code indicates a product (that could have been) purchased by a customer. 

In [36]:
SELECT SUM(UnitPrice)
FROM online_retail.csv
WHERE StockCode <> 'POST';

,sum(UnitPrice)
0,2.452416e+06


Running the above block of code showed that the total 'revenue' was £2,452,415.51. 

1% of this value would be £24,524.15; since this value is lower than the amount spent on postage, "I" spent more than 1% of my revenue on posting the products to customers. 

The next block will focus on the third question I asked: 
3. How many different products were included in the average order? 

This means I will be looking at the mean number of distinct stock codes within a single invoice number. I wanted to start by looking at the number of products in a given invoice, and identify the invoices with the largest number of items in their orders; this acted as some additional data exploration to help contextualise the kind of number I might expect at the end. I am also working under the assumption that a large number of invoices have one product each (note: not 'one item' each!) 

In [37]:
SELECT 
	COUNT(StockCode) AS count_products, 
	InvoiceNo
FROM online_retail.csv
WHERE StockCode <> 'POST' AND StockCode IS NOT NULL
GROUP BY InvoiceNo
ORDER BY count_products DESC
LIMIT (10);

,count_products,InvoiceNo
0,1114,573585
1,749,581219
2,731,581492
3,721,580729
4,705,558475
5,687,579777
6,676,581217
7,675,537434
8,662,580730
9,652,538071


The above block of code shows that invoice number 573585 included 1,114 different products. This seemed like a particularly large number so the next short query finds the number of distinct products sold by 'my' business.

In [42]:
SELECT COUNT(DISTINCT Description)
FROM online_retail.csv
WHERE Description IS NOT NULL;

,count(DISTINCT Description)
0,4223


The above query shows me that 'I' stock 4,223 different products, so a customer placing an order for 1,114 products is reasonable (if not likely in reality). 

To continue with Q3, I went back to trying to find the mean number of different products included in a single invoice. This will require a nested query / use of sub-queries. 

Step 1: find the number of distinct stock codes and the number of invoice numbers
Step 2: use those values to calculate the mean number of products per invoice 

To do this, I've nested the 'intermediate query' (step 1) inside the FROM statement for the 'final query' (step 2). 

In [43]:
SELECT AVG(count_prod / no_orders) AS avg_prod_per_invoice
FROM 
	(SELECT 
	 	COUNT(DISTINCT StockCode) AS count_prod, 
	 	COUNT(DISTINCT InvoiceNo) AS no_orders
	 FROM online_retail.csv
	 WHERE StockCode <> 'POST'
	 GROUP BY InvoiceNo
	 );

,avg_prod_per_invoice
0,20.579001


Running the query abov returns the value 20.58 (2 d.p.) products per invoice. Given that we can assume there were a significant number of invoices with exactly one product type (and the results found earlier), this is a reasonable value. 